In [133]:
import ast
import re
import pandas as pd
import spacy
from pattern.en import pluralize
import en_core_web_sm
nlp = en_core_web_sm.load()

/home/lisa/Code/env/lib/python3.6/site-packages/spacy/util.py:730: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.1.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [115]:
sents = open('mqnli_data', encoding='utf-8').read().split('\n')

In [ ]:
sents_d = []
for x in sents:
    sents_d.append(ast.literal_eval(x))

In [117]:
len(sents_d)

499998

In [19]:
set([x['sentence1'].split(' ')[0] for x in sents_d])

{'every', 'no', 'notevery', 'some'}

In [134]:
def reformat_sentence(sentence):
    new_sentence = []
    negation = False
    for ix,word in enumerate(sentence.split(' ')):
        if word == 'notevery':
            new_sentence.append('not')
            new_sentence.append('every')
        elif word == 'doesnot':
            new_sentence.append("doesn't")
            negation = True
        elif ix==5:
            if negation==True:
                new_sentence.append(nlp(word)[0].lemma_)
            else:
                new_sentence.append(word)
        elif word!='emptystring':
            new_sentence.append(word.lower())
    new_sentence = ' '.join(new_sentence)
    return new_sentence

In [146]:
df = pd.DataFrame(sents_d)

In [ ]:
all_sent1_subj_quant = []
all_sent1_subj_adj = []
all_sent1_obj_quant = []
all_sent1_obj_adj = []
all_sent1_adv = []
all_sent1_polarity = []
all_sent2_subj_quant = []
all_sent2_subj_adj = []
all_sent2_obj_quant = []
all_sent2_obj_adj = []
all_sent2_adv = []
all_sent2_polarity = []
sents1_readable = []
sents2_readable = []

for ix,row in df.iterrows():
    sents1_readable.append(reformat_sentence(row['sentence1']))
    sents2_readable.append(reformat_sentence(row['sentence2']))
    scheme1 = row['sentence1'].split(' ')
    scheme2 = row['sentence2'].split(' ')
    
    sent1_subj_quant = scheme1[0]
    if scheme1[1] == 'emptystring':
        sent1_subj_adj = 'no'
    else:
        sent1_subj_adj = 'yes'
    sent1_obj_quant =scheme1[6]
    if scheme1[7] == 'emptystring':
        sent1_obj_adj = 'no'
    else:
        sent1_obj_adj = 'yes'
    if scheme1[4] == 'emptystring':
        sent1_adv = 'no'
    else:
        sent1_adv = 'yes'
    if scheme1[3] == 'emptystring':
        sent1_polarity = 'aff'
    else:
        sent1_polarity = 'neg'

    sent2_subj_quant = scheme2[0]
    if scheme2[1] == 'emptystring':
        sent2_subj_adj = 'no'
    else:
        sent2_subj_adj = 'yes'
    sent2_obj_quant =scheme2[6]
    if scheme2[7] == 'emptystring':
        sent2_obj_adj = 'no'
    else:
        sent2_obj_adj = 'yes'
    if scheme2[4] == 'emptystring':
        sent2_adv = 'no'
    else:
        sent2_adv = 'yes'
    if scheme2[3] == 'emptystring':
        sent2_polarity = 'aff'
    else:
        sent2_polarity = 'neg'
        
    all_sent1_subj_quant.append(sent1_subj_quant)
    all_sent1_subj_adj.append(sent1_subj_adj)
    all_sent1_obj_quant.append(sent1_obj_quant)
    all_sent1_obj_adj.append(sent1_obj_adj)
    all_sent1_adv.append(sent1_adv)
    all_sent1_polarity.append(sent1_polarity)
    all_sent2_subj_quant.append(sent2_subj_quant)
    all_sent2_subj_adj.append(sent2_subj_adj)
    all_sent2_obj_quant.append(sent2_obj_quant)
    all_sent2_obj_adj.append(sent2_obj_adj)
    all_sent2_adv.append(sent2_adv)
    all_sent2_polarity.append(sent2_polarity)

In [150]:
df['sent1_readable'] = sents1_readable
df['sent2_readable'] = sents2_readable
df['sent1_subj_quant'] = all_sent1_subj_quant
df['sent1_subj_adj'] = all_sent1_subj_adj
df['sent1_obj_quant'] = all_sent1_obj_quant
df['sent1_obj_adj'] = all_sent1_obj_adj
df['sent1_adv'] = all_sent1_adv
df['sent1_polarity'] = all_sent1_polarity
df['sent2_subj_quant'] = all_sent2_subj_quant
df['sent2_subj_adj'] = all_sent2_subj_adj
df['sent2_obj_quant'] = all_sent2_obj_quant
df['sent2_obj_adj'] = all_sent2_obj_adj
df['sent2_adv'] = all_sent2_adv
df['sent2_polarity'] = all_sent2_polarity

In [152]:
df.to_csv('labelled_mqnli.csv')

In [179]:
quants_of_interest = ['no','some']

In [205]:
uniform = df[(df['sent1_subj_quant']==df['sent2_subj_quant'])&(df['sent1_obj_quant']==df['sent2_obj_quant'])]

In [206]:
uniform = uniform[(uniform['sent1_subj_quant'].isin(quants_of_interest))|(uniform['sent1_obj_quant'].isin(quants_of_interest))]

In [207]:
uniform['sent1_subj_quant'].value_counts()

no          11009
some         6096
every        5324
notevery     3062
Name: sent1_subj_quant, dtype: int64

In [208]:
uniform['sent1_obj_quant'].value_counts()

some        8596
no          8387
every       4316
notevery    4192
Name: sent1_obj_quant, dtype: int64

In [209]:
len(uniform)

25491

In [222]:
stage2_indices = []
subj_some_cnt = 0
subj_no_cnt = 0
obj_some_cnt = 0
obj_no_cnt = 0
subj_some_ix = []
subj_no_ix = []
obj_some_ix = []
obj_no_ix = []
for ix,row in uniform.iterrows():
    if row['sent1_obj_quant'] == 'some' and obj_some_cnt<4000:
        stage2_indices.append(ix)
        obj_some_cnt += 1
        obj_some_ix.append(ix)
    elif row['sent1_obj_quant'] == 'no' and obj_no_cnt<4000:
        stage2_indices.append(ix)
        obj_no_cnt += 1
        obj_no_ix.append(ix)
    elif row['sent1_subj_quant'] == 'some' and subj_some_cnt<4000:
        stage2_indices.append(ix)
        subj_some_cnt += 1
        subj_some_ix.append(ix)
    elif row['sent1_subj_quant'] == 'no' and subj_no_cnt<4000:
        stage2_indices.append(ix)
        subj_no_cnt += 1   
        subj_no_ix.append(ix)

In [223]:
len(stage2_indices)

16000

In [224]:
stage1_df = df[~df.index.isin(stage2_indices)]

In [225]:
len(stage1_df)

483998

In [230]:
not_indices = []
neg_count = 0
for ix,row in stage1_df.iterrows():
    if neg_count>7999: break
    if (row['sent1_polarity']=='neg' or row['sent2_polarity']=='neg'):
        not_indices.append(ix)
        neg_count+=1

In [233]:
stage1_df = stage1_df[~stage1_df.index.isin(not_indices)]

In [234]:
len(stage1_df)

475998

In [235]:
stage1_df.to_csv('stage1_training_data.csv')

In [236]:
stage2_indices = stage2_indices + not_indices

In [238]:
stage2_indices = sorted(stage2_indices)

In [255]:
stage2_df = uniform[uniform.index.isin(stage2_indices)]

In [256]:
stage2_quant_s1 = []
stage2_quant_s2 = []
stage2_quant_s1_readable = []
stage2_quant_s2_readable = []
quant_subj = []
quant_obj = []
for ix,row in stage2_df.iterrows():
    s1 = row['sentence1'].split(' ')
    s2 = row['sentence2'].split(' ')
    if ix in subj_some_ix:
        new_s1 = ' '.join(['[MANY]']+[s1[1]]+[pluralize(s1[2])]+s1[3:])
        new_s2 = ' '.join(['[MANY]']+[s2[1]]+[pluralize(s2[2])]+s2[3:])
        quant_subj.append('[MANY]')
        quant_obj.append(row['sent1_obj_quant'])
    elif ix in subj_no_ix:
        new_s1 = ' '.join(['[FEW]']+[s1[1]]+[pluralize(s1[2])]+s1[3:])
        new_s2 = ' '.join(['[FEW]']+[s2[1]]+[pluralize(s2[2])]+s2[3:])
        quant_subj.append('[FEW]')
        quant_obj.append(row['sent1_obj_quant'])
    elif ix in obj_some_ix:
        new_s1 = ' '.join(s1[:6]+['[MANY]']+[s1[7]]+[pluralize(s1[8])])
        new_s2 = ' '.join(s2[:6]+['[MANY]']+[s2[7]]+[pluralize(s2[8])])
        quant_obj.append('[MANY]')
        quant_subj.append(row['sent1_subj_quant'])
    elif ix in obj_no_ix:
        new_s1 = ' '.join(s1[:6]+['[FEW]']+[s1[7]]+[pluralize(s1[8])])
        new_s2 = ' '.join(s2[:6]+['[FEW]']+[s2[7]]+[pluralize(s2[8])])
        quant_obj.append('[FEW]')
        quant_subj.append(row['sent1_subj_quant'])
    elif ix in not_indices:
        new_s1 = ' '.join(s1[:3]+['[NOT]']+s1[4:])
        new_s2 = ' '.join(s2[:3]+['[NOT]']+s2[4:])
        quant_obj.append(row['sent1_obj_quant'])
        quant_subj.append(row['sent1_subj_quant'])
        
    stage2_quant_s1.append(new_s1)
    stage2_quant_s2.append(new_s2)
    stage2_quant_s1_readable.append(reformat_sentence(new_s1).replace('[many]','[MANY]').replace('[few]','[FEW]').replace('[not]','[NOT]'))
    stage2_quant_s2_readable.append(reformat_sentence(new_s2).replace('[many]','[MANY]').replace('[few]','[FEW]').replace('[not]','[NOT]'))

In [ ]:
stage2_df['sentence1'] = stage2_quant_s1
stage2_df['sentence2'] = stage2_quant_s2
stage2_df['sent1_readable'] = stage2_quant_s1_readable
stage2_df['sent2_readable'] = stage2_quant_s2_readable
stage2_df['sent1_subj_quant'] = quant_subj
stage2_df['sent2_subj_quant'] = quant_subj
stage2_df['sent1_obj_quant'] = quant_obj
stage2_df['sent2_obj_quant'] = quant_obj

In [262]:
stage2_df.to_csv('stage2_training_data.csv')